## Sonification of Piano Rolls
Another symbolic music representation is often referred to as piano roll representation. Within these representations, note events are described by different attributes: **start**, **duration**, **pitch**, **velocity** and **label**. In this notebook we illustrate the sonification of piano rolls usting the ```libsoni.core.pianoroll``` module.

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from IPython import display as ipd

from libsoni.core.pianoroll import sonify_pianoroll_additive_synthesis, \
                                   sonify_pianoroll_fm_synthesis, \
                                   sonify_pianoroll_clicks, \
                                   sonify_pianoroll_sample, \
                                   visualize_pianoroll

from libsoni.core.tse import sonify_tse_clicks
from libsoni.util.utils import mix_sonification_and_original, get_preset

Fs = 22050

## Simple Scenario: C Major Triad

To start with a simple example, let's look at a **C major triad**.
<img src="figures/demo_f0/C-Dur-DM.png" alt="C-Major-Triad" width="250">
The pitches of the corresponding to the notes are:

| Note |      Pitch     |
|------|----------------|
| C4   |       60       |
| E4   |       64       |
| G4   |       67       |
| C5   |       72       |

### Creating a Piano Roll
In order to obtain a piano roll representation of the notes of the C major triad shown above, we have to define the temporal properties of the notes by their starting points and durations. Let's choose the note **C4** to be played at 0.25 seconds and the following notes each for one second after the preceding start. All notes should be played for 0.8 seconds.
For the pitches, we adopt the column from the table above, and we set the velocity to 1. The attribute **label** is useful in a context, where different note events come from different instruments, so we omit this in our simple scenario for the moment.

In [ ]:

cmaj_time_positions = [0.25, 1.25, 2.25, 3.25]
cmaj_durations = [0.8, 0.8, 0.8, 0.8]
cmaj_pitches = [60, 64, 67, 72]
cmaj_velocities = [1.0, 1.0, 1.0, 1.0]

c_maj_df = pd.DataFrame({'start': cmaj_time_positions,
                         'pitch': cmaj_pitches,
                         'duration': cmaj_durations,
                         'velocity': cmaj_velocities,
                         'label': 'piano'})
ipd.display(c_maj_df)

visualize_pianoroll(c_maj_df, title='Piano Roll: C Major Triad');

### Sonification of Piano Rolls
For sonifying piano roll representations, the ```libsoni.core.pianoroll``` module offers different methods. For the beginning, we sonify the piano roll example above using so-called **additive synthesis**.

In [ ]:
# Sonification using libsoni
sonified_cmaj_as = sonify_pianoroll_additive_synthesis(pianoroll_df=c_maj_df)

print('Sonified Piano Roll Representation of C Major Triad:')
ipd.display(ipd.Audio(sonified_cmaj_as, rate=Fs))

### A more Interesting but still Simple Scenario
Let's make the above example a little more interesting, and vary the lengths and velocities of the notes.

In [ ]:
c_maj_modified_df = c_maj_df.copy()
c_maj_modified_df['velocity'] = [0.2, 0.4, 0.6, 0.8]
c_maj_modified_df['duration'] = [0.8, 0.6, 0.4, 0.2]

# Sonification using libsoni
sonified_cmaj_modified_as = sonify_pianoroll_additive_synthesis(pianoroll_df=c_maj_modified_df)

print('Sonified Modified Piano Roll Representation of C Major Triad:')
ipd.display(ipd.Audio(sonified_cmaj_modified_as, rate=Fs))
visualize_pianoroll(c_maj_modified_df, title='Modifed Piano Roll: C Major Triad');

### Customizing the Sonification - Additive Synthesis
In order to change the sonification tonally, so-called partials as well as their amplitudes can be specified. Let's say we want to use the fundamental frequency (1) as well as two times, three times and four times the fundamental frequency as partials for sonification. For the amplitudes we want to set 1, 1/2, 1/3, 1/4.

| Partials           | Amplitudes       |
|--------------------|------------------|
| 1                  | 1                |
| 2                  | 1/2              |
| 3                  | 1/3              |
| 4                  | 1/4              |

In [ ]:
partials = [1, 2, 3, 4]
amplitudes_partials = [1, 1/2, 1/3, 1/4] 

print('Sonified C Major Scale with Additive Synthesis:')
ipd.display(ipd.Audio(sonified_cmaj_modified_as, rate=Fs))


sonified_cmaj_modified_as_modified = sonify_pianoroll_additive_synthesis(pianoroll_df=c_maj_modified_df,
                                                                         partials=[1, 2, 3],
                                                                         partials_amplitudes=[0.5, 0.25, 0.25])
print('Sonified C Major Scale with Additive Synthesis, using different partials:')
ipd.display(ipd.Audio(sonified_cmaj_modified_as_modified, rate=Fs))

### Customizing the Sonification - Frequency Modulation Synthesis
Another method for sonification is based on so-called frequency modulation (FM) synthesis. In this synthesis method, the sound is shaped by two parameters: the modulation frequency and the modulation index. For a detailed explanation of the synthesis, please refer to X. In the following we show a few examples for different parameters.

In [ ]:
# Define different FM Synthesis parameters
modulation_frequency_factors = [0.5, 1, 2, 3]
modulation_indices = [0.5, 2, 4, 10]

for modulation_frequency_factor, modulation_index in zip(modulation_frequency_factors, modulation_indices):

    sonified_cmaj_fm = sonify_pianoroll_fm_synthesis(pianoroll_df=c_maj_modified_df,
                                                     modulation_frequency_factor=modulation_frequency_factor,
                                                     modulation_index=modulation_index)
    print(f'Sonified C Major Scale with FM-Synthesis using Modulation Frequency Factor {modulation_frequency_factor} and Modulation Index {modulation_index}:')
    ipd.display(ipd.Audio(sonified_cmaj_fm, rate=Fs))

### Sonification of Piano Rolls using Colored Clicks
We can also use colored clicks for the sonification. In case only the onsets of the notes should be sonified, in other words, the pitch, duration and velocity information should be omitted, we can use the ```sonify_tse_clicks```function from ```libsoni.core.tse```.

In [ ]:
visualize_pianoroll(c_maj_modified_df, title='Piano Roll: C Major Triad');

sonified_cmaj_clicks_onsets_only = sonify_tse_clicks(time_positions=c_maj_modified_df['start'].to_numpy())
print('Sonified onsets with clicks:')
ipd.display(ipd.Audio(sonified_cmaj_clicks_onsets_only, rate=Fs))


sonified_cmaj_clicks = sonify_pianoroll_clicks(pianoroll_df=c_maj_modified_df)
print('Sonified C Major Scale with clicks:')
ipd.display(ipd.Audio(sonified_cmaj_clicks, rate=Fs))

### Combining Sonifications
We can also combine different sonifications, like clicks and additive synthesis:

In [ ]:
cmaj_additive = sonify_pianoroll_additive_synthesis(pianoroll_df=c_maj_modified_df,
                                                    partials=[0.5, 1, 2],
                                                    partials_amplitudes=[0.33, 0.66, 0.33])

cmaj_clicks = sonify_pianoroll_clicks(pianoroll_df=c_maj_modified_df)

print('Sonification using additive synthesis and clicks')
ipd.display(ipd.Audio(cmaj_additive + cmaj_clicks*10, rate=Fs))

## Scenario: *Bach Fugue in C Major*, BWV 846, by *J. S. Bach*
The Fugue in C Major, BWV 846, by Johann Sebastian Bach presents a concise theme that winds through various voices, creating overlapping and interweaving textures. In the following scenario, we get the piano roll representation by an annotation file and sonify it with various methods as shown above.

In [ ]:
bach_df = pd.read_csv(os.path.join('data_csv',
                                   'demo_pianoroll',
                                   'FMP_C1_F12_Bach_BWV846_Sibelius-Tracks.csv'),delimiter=';')

visualize_pianoroll(bach_df, figsize=(10, 7), colors='gist_rainbow', title='Piano Roll: Fugue in C Major, J. S. Bach');

# Additive Synthesis
print('Sonified with Additive Synthesis:')

sonified_bach_as = sonify_pianoroll_additive_synthesis(pianoroll_df=bach_df,
                                                       partials=[1, 2, 3],
                                                       partials_amplitudes=[0.5, 0.25, 0.25])
ipd.display(ipd.Audio(sonified_bach_as, rate=Fs))

# Frequency Modulation Synthesis
print('Sonified with Frequency Modulation Synthesis:')

sonified_bach_fm = sonify_pianoroll_fm_synthesis(pianoroll_df=bach_df,
                                                 modulation_frequency_factor=2,
                                                 modulation_index=1)
ipd.display(ipd.Audio(sonified_bach_fm, rate=Fs))

# Colored Clicks
print('Sonified with Colored Clicks:')

sonification_bach_clicks = sonify_pianoroll_clicks(pianoroll_df=bach_df)

ipd.display(ipd.Audio(sonification_bach_clicks, rate=Fs))

# Clicks and Frequency Modulation Synthesis
print('Sonification using Frequency Modulation Synthesis and Colored Clicks:')

ipd.display(ipd.Audio(sonified_bach_fm + sonification_bach_clicks, rate=Fs))